In [11]:
import pandas as pd
import numpy as np
import os

In [12]:
# Caminho para o Excel original
xlsx_path = "../data/labels/OSATS.xlsx"

# Carregar o Excel
df = pd.read_excel(xlsx_path)

# Visualizar as primeiras linhas
df.head()

,STUDENT,GROUP,TIME,SUTURES,INVESTIGATOR,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE
0,AHO729,E-LEARNING,PRE,1.0,A,P54M,2,1,2,2,1,2,2,1,13
1,AHO729,E-LEARNING,PRE,1.0,B,P54M,2,1,3,1,2,1,2,1,13
2,AHO729,E-LEARNING,PRE,1.0,C,P54M,2,1,1,1,1,1,1,1,9
3,AHO729,E-LEARNING,POST,4.5,A,M45P,4,4,4,3,3,4,3,3,28
4,AHO729,E-LEARNING,POST,4.5,B,M45P,2,3,4,3,3,4,3,3,25


In [13]:
# Converter colunas relevantes para números (caso haja erros de tipo)
numeric_cols = [col for col in df.columns if 'OSATS_' in col or col == 'GLOBA_RATING_SCORE']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Agrupar por VIDEO e tirar a média
df_grouped = df.groupby("VIDEO")[numeric_cols].mean()

# Arredondar para inteiros, como esperado no desafio
df_grouped_rounded = df_grouped.round(0).astype(int)

# Visualizar
df_grouped_rounded.head()

,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE
VIDEO,,,,,,,,,
A31H,1,2,2,2,2,2,2,1,14
A36O,3,1,2,1,1,1,1,1,12
A40E,3,2,2,2,2,2,2,2,16
A41X,5,4,4,5,4,4,4,4,34
A59Z,2,1,2,1,1,1,1,1,11


In [14]:
def score_to_class(score):
    if score <= 15:
        return 0  # Novice
    elif score <= 23:
        return 1  # Intermediate
    elif score <= 31:
        return 2  # Proficient
    else:
        return 3  # Expert

# Aplicar função à coluna GLOBA_RATING_SCORE
df_grouped_rounded["GRS_CLASS"] = df_grouped_rounded["GLOBA_RATING_SCORE"].apply(score_to_class)

# Visualizar
df_grouped_rounded[["GLOBA_RATING_SCORE", "GRS_CLASS"]].head()

,GLOBA_RATING_SCORE,GRS_CLASS
VIDEO,,
A31H,14,0
A36O,12,0
A40E,16,1
A41X,34,3
A59Z,11,0


In [15]:
# CSV de GRS
task1 = df_grouped_rounded[["GRS_CLASS"]].reset_index()
task1.columns = ["VIDEO", "GRS"]

# Guardar
output_path_1 = "../data/labels/labels_task1.csv"
task1.to_csv(output_path_1, index=False)

print(f"✅ Task 1 CSV salvo em: {output_path_1}")
task1.head()

✅ Task 1 CSV salvo em: ../data/labels/labels_task1.csv


,VIDEO,GRS
0,A31H,0
1,A36O,0
2,A40E,1
3,A41X,3
4,A59Z,0


In [16]:
# Selecionar colunas OSATS
osats_cols = [col for col in df_grouped_rounded.columns if col.startswith("OSATS_")]

# CSV de OSATS
task2 = df_grouped_rounded[osats_cols].reset_index()
task2.columns = ["VIDEO"] + osats_cols

# Guardar
output_path_2 = "../data/labels/labels_task2.csv"
task2.to_csv(output_path_2, index=False)

print(f"✅ Task 2 CSV salvo em: {output_path_2}")
task2.head()

✅ Task 2 CSV salvo em: ../data/labels/labels_task2.csv


,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY
0,A31H,1,2,2,2,2,2,2,1
1,A36O,3,1,2,1,1,1,1,1
2,A40E,3,2,2,2,2,2,2,2
3,A41X,5,4,4,5,4,4,4,4
4,A59Z,2,1,2,1,1,1,1,1


In [17]:
print("✅ Processamento completo.")
print(f"{len(task1)} vídeos processados.")
print(f"GRS classes: {task1['GRS'].value_counts().sort_index().to_dict()}")

✅ Processamento completo.
314 vídeos processados.
GRS classes: {0: 113, 1: 69, 2: 113, 3: 19}
